This notebook is aimed at looked at the allele analysis output a bit more interms of what are the alleles. What is missing? What is the GO term enrichment in those if any? How much are effectors?....
The input is based on the following ideas:


the idea is to label alleles, non_allelic 'orthologs' singeltons, and single nuclear genes.
* alleles are bast on proteinortho -synteny flag
* non_allelic 'orthologs' are genes that cluster on the protein level but are not allelic
* singeltons are genes without ‘ortholog’ in the other haplotype
* single haplotype genes are genes that have no gene to genome blast hit e_value = 1e-2 and are not in a homozygous region.

Final analysis is based on Pst_104E_v12_missing_allele_QC_v03.ipynb and Pst_104E_v12_defining_alleles_v03.ipynb in folder 
/home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/Pst_104E_v12/allele_analysis/no_alleles_proteinortho_graph516_QC_Qcov80_PctID70_evalue01 and alleles_proteinortho_graph516


In [1]:
%matplotlib inline

In [130]:
import pandas as pd
import os
import re
from Bio import SeqIO
from Bio import SeqUtils
import pysam
from Bio.SeqRecord import SeqRecord
from pybedtools import BedTool
import numpy as np
import pybedtools
import time
import matplotlib.pyplot as plt
import sys
import subprocess
import shutil
from Bio.Seq import Seq
import pysam
from Bio import SearchIO
import json
import glob
import scipy.stats as stats

In [3]:
pwh_set = []
def pwh_filter (q_contig, pwh_set=pwh_set):
    '''Checks if contig belongs to the primary with haplotig set.'''
    if q_contig in pwh_set:
        return True
    else:
        return False

In [4]:
def on_primary_contig (q_contig):
    '''Quick function that checks if query is on primary contig or not'''
    if q_contig.startswith('hcontig'):
        return False
    elif q_contig.startswith('pcontig'):
        return True
    else:
        print('Contig annotation needs to start with hcontig or pcontig')

In [127]:
#Define the PATH
BASE_AA_PATH = '/home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/Pst_104E_v12'
BASE_A_PATH = '/home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/032017_assembly'
BLAST_RESULT_PATH = os.path.join(BASE_AA_PATH,'allele_analysis' )
ALLELE_PATH =os.path.join(BASE_AA_PATH ,'allele_analysis/alleles_proteinortho_graph516')
ALLELE_QC_PATH = os.path.join(BASE_AA_PATH, 'allele_analysis', \
                              'no_alleles_proteinortho_graph516_QC_Qcov80_PctID70_evalue01')
LIST_PATH = os.path.join(BASE_AA_PATH, 'enrichment_analysis', 'lists')
POST_ALLELE_PATH = os.path.join(BASE_AA_PATH, 'post_allele_analysis')
OUT_PATH = os.path.join(POST_ALLELE_PATH, 'proteinortho_graph516_QC_Qcov80_PctID70_evalue01')
if not os.path.exists(POST_ALLELE_PATH):
    os.mkdir(POST_ALLELE_PATH)
if not os.path.exists(OUT_PATH):
    os.mkdir(OUT_PATH)
ph_gene_fasta = os.path.join(BASE_A_PATH , 'Pst_104E_v12_ph_ctg.anno.gene.fa')

In [6]:
#get all the alleles in as they are not filtered by QCov and PctID but simply taken straight from the 
# proteinortho
allele_header = ['p_gene', 'h_gene']
a_overlap_df = pd.read_csv(os.path.join(ALLELE_PATH, \
                    'Pst_104E_v12_p_ctg.h_contig_overlap.alleles'), sep='\t', header = None,\
                           names=allele_header)
a_overlap_df['Linkage'] = 'h_contig_overlap'
a_no_roverlap_df = pd.read_csv(os.path.join(ALLELE_PATH, \
                    'Pst_104E_v12_p_ctg.no_respective_h_contig_overlap.alleles'), sep='\t', header = None,\
                           names=allele_header)
a_no_roverlap_df['Linkage'] = 'no_r_overlap'
a_no_soverlap_df = pd.read_csv(os.path.join(ALLELE_PATH, \
                    'Pst_104E_v12_p_ctg.no_specific_h_contig_overlap.alleles'), sep='\t', header = None,\
                           names=allele_header)
a_no_soverlap_df['Linkage'] = 'no_s_overlap'
allele_df = pd.concat([a_overlap_df, a_no_roverlap_df, a_no_soverlap_df], axis=0)

In [15]:
#copy over allele files to OUTPATH
os.chdir(ALLELE_PATH)
!cp Pst_104E_v12_p_ctg.h_contig_overlap.alleles {OUT_PATH}/.
!cp Pst_104E_v12_p_ctg.no_respective_h_contig_overlap.alleles {OUT_PATH}/.
!cp Pst_104E_v12_p_ctg.no_specific_h_contig_overlap.alleles {OUT_PATH}/. 
!cat Pst_104E_v12_p_ctg.h_contig_overlap.alleles Pst_104E_v12_p_ctg.no_respective_h_contig_overlap.alleles Pst_104E_v12_p_ctg.no_specific_h_contig_overlap.alleles > {OUT_PATH}/Pst_104E_v12_p_ctg.all.alleles

In [26]:
#get the blast dataframe and the QC dataframe
allele_blast_df = pd.read_csv(os.path.join(BLAST_RESULT_PATH, 'Pst_104E_v12_p_ctg.Pst_104E_v12_h_ctg.0.001.blastp.outfmt6.allele_analysis'), sep='\t')
allele_QC_df = pd.read_csv(os.path.join(ALLELE_QC_PATH, 'Pst_104E_v12_ph_ctg.no_alleles_QC.Qcov80.PctID70.df'), sep='\t')

/home/benjamin/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [44]:
allele_QC_df = pd.read_csv(os.path.join(ALLELE_QC_PATH, 'Pst_104E_v12_ph_ctg.no_alleles_QC.Qcov80.PctID70.df'), sep='\t')
allele_QC_df = allele_QC_df[~((allele_QC_df.Query.isin(allele_df.p_gene))|(allele_QC_df.Query.isin(allele_df.h_gene)))]


/home/benjamin/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [47]:
#quick check if all the numbers of genes add up
len(allele_df.p_gene.unique())+ len(allele_df.h_gene.unique())+len(allele_QC_df.Query.unique()) == 30249

True

In [53]:
#now write out all no_allele_orthologs
no_allele_orthologs_fn = os.path.join(OUT_PATH, 'Pst_104E_ph_ctg.no_alleles_orthologs')
no_allele_orthologs = allele_QC_df[allele_QC_df.singeltons == False]["Query"].unique()
tmp_fh = open(no_allele_orthologs_fn, 'w')
for _id in allele_QC_df[allele_QC_df.singeltons == False]["Query"].unique():
    print(_id, file=tmp_fh)
tmp_fh.close()

In [97]:
#write out singeltons
singeltons_fh = os.path.join(OUT_PATH, 'Pst_104E_ph_ctg.singletons')
singeltons = allele_QC_df[(allele_QC_df.singeltons == True) & (allele_QC_df.ph_p_homo_region != True)]["Query"].unique()
tmp_fh = open(singeltons_fh , 'w')
for _id in singeltons:
    print(_id, file=tmp_fh)
tmp_fh.close()

In [98]:
#write out single nuclear genes
shg_df = allele_QC_df[(allele_QC_df.singeltons == True) & (allele_QC_df.gene_on_genome_blast_hit != True)\
            &(allele_QC_df.ph_p_homo_region != True)].copy()
shg_fh = os.path.join(OUT_PATH, 'Pst_104E_ph_ctg.single_haplotype_genes')
shg = shg_df.Query.unique()
tmp_fh = open(shg_fh, 'w')
for _id in shg:
    print(_id, file=tmp_fh)
tmp_fh.close()

In [99]:
#now read in some types of genes
gene_group_list = ['BUSCOS', 'EFFECTORS', 'HAUSTORIA', 'EFFECTORP']
p_gene_dict = {}
h_gene_dict = {}
p_gene_dict['BUSCOS'] = pd.read_csv(os.path.join(LIST_PATH, 'Pst_104E_v12_p_busco.list')\
                                    , header=None)[0].tolist()
p_gene_dict['EFFECTORS'] = pd.read_csv(os.path.join(LIST_PATH, 'Pst_104E_v12_p_effector.list')\
                                    , header=None)[0].tolist()
p_gene_dict['HAUSTORIA'] = pd.read_csv(os.path.join(LIST_PATH, 'Pst_104E_v12_cluster_8.list')\
                                    , header=None)[0].tolist()
p_gene_dict['EFFECTORP'] = pd.read_csv(os.path.join(LIST_PATH, 'Pst_104E_v12_p_effectorp.list')\
                                    , header=None)[0].tolist()
h_gene_dict['BUSCOS'] = pd.read_csv(os.path.join(LIST_PATH, 'Pst_104E_v12_h_busco.list')\
                                    , header=None)[0].tolist()
h_gene_dict['EFFECTORS'] = pd.read_csv(os.path.join(LIST_PATH, 'Pst_104E_v12_h_effector.list')\
                                    , header=None)[0].tolist()
h_gene_dict['HAUSTORIA'] = pd.read_csv(os.path.join(LIST_PATH, 'Pst_104E_v12_h_cluster_15.list')\
                                    , header=None)[0].tolist()
h_gene_dict['EFFECTORP'] = pd.read_csv(os.path.join(LIST_PATH, 'Pst_104E_v12_h_effectorp.list')\
                                    , header=None)[0].tolist()
#now get pwh and pwoh
pwh_list = pd.read_csv(os.path.join(BASE_A_PATH, 'Pst_104E_v12_pwh.txt')\
                                , header=None)[0].tolist()
pwoh_list = pd.read_csv(os.path.join(BASE_A_PATH, 'Pst_104E_v12_pwoh.txt')\
                                , header=None)[0].tolist()

In [100]:
#cause we are a bit lazy an like notebooks here we go looking for stuff
os.chdir(BASE_A_PATH)
print("Effectors on pwh contigs")
!cat Pst_104E_v12_pwh.txt | xargs -I X grep -w 'X' {LIST_PATH}/Pst_104E_v12_p_effector.gene.gff3 | wc -l 

Effectors on pwh contigs
1523


In [101]:
print("Effectors on pwoh contigs")
!cat Pst_104E_v12_pwoh.txt | xargs -I X grep -w 'X' {LIST_PATH}/Pst_104E_v12_p_effector.gene.gff3 | wc -l 

Effectors on pwoh contigs
49


In [102]:
print("BUSCOS on pwoh contigs")
!cat Pst_104E_v12_pwoh.txt | xargs -I X grep -w 'X' {LIST_PATH}/Pst_104E_v12_p_busco.gene.gff3 | wc -l 

BUSCOS on pwoh contigs
49


In [103]:
print("BUSCOS on pwh contigs")
!cat Pst_104E_v12_pwh.txt | xargs -I X grep -w 'X' {LIST_PATH}/Pst_104E_v12_p_busco.gene.gff3 | wc -l 

BUSCOS on pwh contigs
1395


In [104]:
print("BUSCOS on hcontigs")
!wc -l {LIST_PATH}/Pst_104E_v12_h_busco.gene.gff3

BUSCOS on hcontigs
1293 /home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/Pst_104E_v12/enrichment_analysis/lists/Pst_104E_v12_h_busco.gene.gff3


In [105]:
#now see if there is an enrichment in GO terms associated with alleles
def orphan_analysis(_orphan_list):
    orphan_effectors = 0
    orphan_effectors_h = 0
    orphan_effectors_pwh = 0
    orphan_effectors_pwoh =0
    orphan_busco = 0
    orphan_busco_h = 0
    orphan_busco_pwh = 0
    orphan_busco_pwoh = 0
    for x in _orphan_list:
        if x in p_gene_dict['BUSCOS']:
            contig = re.search(r'[hp][^.]*',x).group()
            if pwh_filter(contig, pwh_list) == True:
                orphan_busco_pwh += 1
            else:
                orphan_busco_pwoh += 1
        if x in h_gene_dict['BUSCOS']:
            orphan_busco_h += 1
        #now same for effectors    
        if x in p_gene_dict['EFFECTORS']:
            contig = re.search(r'[hp][^.]*',x).group()
            if pwh_filter(contig, pwh_list) == True:
                orphan_effectors_pwh += 1
            else:
                orphan_effectors_pwoh += 1
            
        if x in h_gene_dict['EFFECTORS']:
            orphan_effectors_h += 1      
    orphan_effectors = orphan_effectors_h + orphan_effectors_pwh + orphan_effectors_pwoh
    orphan_busco = orphan_busco_h +  orphan_busco_pwh +  orphan_busco_pwoh
    print('Out of %i orphan genes we have %i orphan buscos and %i orphan effectors.'\
          %(len(_orphan_list), orphan_busco, orphan_effectors))
    print('On haplotigs. Out of %i orphan genes we have %i orphan buscos and %i orphan effectors.'\
         %(len([x for x in _orphan_list if 'hcontig' in x]), orphan_busco_h, orphan_effectors_h))
    print('On pwh. Out of %i orphan genes we have %i orphan buscos and %i orphan effectors.'\
         %([pwh_filter(re.search(r'[hp][^.]*',x).group(), pwh_list) for x in _orphan_list].count(True), orphan_busco_pwh, orphan_effectors_pwh))
    print('On pwoh. Out of %i orphan genes we have %i orphan buscos and %i orphan effectors.'\
         %([pwh_filter(re.search(r'[hp][^.]*',x).group(), pwh_list) for x in _orphan_list if 'pcontig' in x].count(False), orphan_busco_pwoh, orphan_effectors_pwoh))

In [166]:
print("Number of genes without clear allele:")
len(allele_QC_df[allele_QC_df.ph_p_homo_region != True]['Query'].unique())

Number of genes without clear allele:


7912

In [106]:
orphan_analysis(shg)

Out of 1506 orphan genes we have 125 orphan buscos and 120 orphan effectors.
On haplotigs. Out of 160 orphan genes we have 6 orphan buscos and 5 orphan effectors.
On pwh. Out of 1149 orphan genes we have 93 orphan buscos and 98 orphan effectors.
On pwoh. Out of 197 orphan genes we have 26 orphan buscos and 17 orphan effectors.


In [107]:
orphan_analysis(singeltons)

Out of 7029 orphan genes we have 234 orphan buscos and 453 orphan effectors.
On haplotigs. Out of 2931 orphan genes we have 57 orphan buscos and 153 orphan effectors.
On pwh. Out of 3700 orphan genes we have 147 orphan buscos and 270 orphan effectors.
On pwoh. Out of 398 orphan genes we have 30 orphan buscos and 30 orphan effectors.


In [108]:
#set the coverage limits
QCov_limit = 80
PctID_limit = 70
#now get all the ids for which we have hits above the cut off and are singletons
tmp_df = allele_QC_df[allele_QC_df.Target != 'False'] #filter out no hits
tmp_df[['QCov', 'PctID']] = tmp_df[['QCov', 'PctID']].apply(pd.to_numeric) #make numeric
#get ids of stuff above the cut off
tmp_df_ids = tmp_df[(tmp_df.QCov >= QCov_limit) & (tmp_df.PctID >= PctID_limit)]['Query'].unique()
#filter those out and everything that is not a singleton
tmp_df = allele_QC_df[~((allele_QC_df.Query.isin(tmp_df_ids)) | (allele_QC_df.singeltons == False))].copy()

/home/benjamin/anaconda3/lib/python3.5/site-packages/pandas/core/frame.py:2352: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [109]:
#now filter dataframe on different other issues
out_filter = (tmp_df.exn_asso_contig != True) & (tmp_df.exn_no_asso_contig != True) & \
            (tmp_df.ph_p_homo_region != True)
queries_out_filtered = tmp_df[out_filter]['Query'].unique()
#now see if there is an enrichment in GO terms associated with alleles
orphan_analysis(queries_out_filtered)
#now print this out again into the ALLELE_QC folder
fn_short = 'Pst_104E_v12_ph_ctg.no_alleles_postQC.PctID%i_QCov%i.no_exonerate' % (PctID_limit, QCov_limit)
fn_short_p = 'Pst_104E_v12_p_ctg.no_alleles_postQC.PctID%i_QCov%i.no_exonerate' % (PctID_limit, QCov_limit)
fn_short_h = 'Pst_104E_v12_h_ctg.no_alleles_postQC.PctID%i_QCov%i.no_exonerate' % (PctID_limit, QCov_limit)
fn = os.path.join(ALLELE_QC_PATH, fn_short)
fn_list_h = os.path.join(LIST_PATH, fn_short_h)
fn_list_p = os.path.join(LIST_PATH, fn_short_p)
fh_list_p = open(fn_list_p, 'w')
fh_list_h = open(fn_list_h, 'w')
fh = open(fn, 'w')
for gene in queries_out_filtered:
    if 'pcontig' in gene:
        print(gene, file=fh_list_p)
    if 'hcontig' in gene:
        print(gene, file=fh_list_h)
    print(gene, file=fh)
fh.close()
fh_list_p.close()
fh_list_h.close()

Out of 3191 orphan genes we have 180 orphan buscos and 207 orphan effectors.
On haplotigs. Out of 942 orphan genes we have 26 orphan buscos and 34 orphan effectors.
On pwh. Out of 1998 orphan genes we have 126 orphan buscos and 152 orphan effectors.
On pwoh. Out of 251 orphan genes we have 28 orphan buscos and 21 orphan effectors.


In [110]:
#now filter dataframe on different other issues
out_filter =  (tmp_df.ph_p_homo_region != True)
queries_out_filtered = tmp_df[out_filter]['Query'].unique()
#now see if there is an enrichment in GO terms associated with alleles
orphan_analysis(queries_out_filtered)
#now print this out again into the ALLELE_QC folder
fn_short = 'Pst_104E_v12_ph_ctg.no_alleles_postQC.PctID%i_QCov%i.no_homo' % (PctID_limit, QCov_limit)
fn_short_p = 'Pst_104E_v12_p_ctg.no_alleles_postQC.PctID%i_QCov%i.no_homo' % (PctID_limit, QCov_limit)
fn_short_h = 'Pst_104E_v12_h_ctg.no_alleles_postQC.PctID%i_QCov%i.no_homo' % (PctID_limit, QCov_limit)
fn = os.path.join(ALLELE_QC_PATH, fn_short)
fn_list_h = os.path.join(LIST_PATH, fn_short_h)
fn_list_p = os.path.join(LIST_PATH, fn_short_p)
fh_list_p = open(fn_list_p, 'w')
fh_list_h = open(fn_list_h, 'w')
fh = open(fn, 'w')
for gene in queries_out_filtered:
    if 'pcontig' in gene:
        print(gene, file=fh_list_p)
    if 'hcontig' in gene:
        print(gene, file=fh_list_h)
    print(gene, file=fh)
fh.close()
fh_list_p.close()
fh_list_h.close()

Out of 3991 orphan genes we have 190 orphan buscos and 303 orphan effectors.
On haplotigs. Out of 1335 orphan genes we have 31 orphan buscos and 83 orphan effectors.
On pwh. Out of 2393 orphan genes we have 131 orphan buscos and 198 orphan effectors.
On pwoh. Out of 263 orphan genes we have 28 orphan buscos and 22 orphan effectors.


In [111]:
#now make new filter for the following
out_filter = (tmp_df.exn_asso_contig != True) & (tmp_df.exn_no_asso_contig != True) & \
            (tmp_df.ph_p_homo_region == True)
queries_out_filtered = tmp_df[out_filter]['Query'].unique()
#now see if there is an enrichment in GO terms associated with alleles
orphan_analysis(queries_out_filtered)

Out of 534 orphan genes we have 62 orphan buscos and 44 orphan effectors.
On haplotigs. Out of 0 orphan genes we have 0 orphan buscos and 0 orphan effectors.
On pwh. Out of 474 orphan genes we have 57 orphan buscos and 43 orphan effectors.
On pwoh. Out of 60 orphan genes we have 5 orphan buscos and 1 orphan effectors.


In [112]:
#now see if there is an enrichment in GO terms associated with alleles
#now make new filter for the following
out_filter = (tmp_df.ph_p_homo_region == True) 
queries_out_filtered = tmp_df[out_filter]['Query'].unique()
#now see if there is an enrichment in GO terms associated with alleles
orphan_analysis(queries_out_filtered)
#now print this out again into the ALLELE_QC folder
fn_short = 'Pst_104E_v12_ph_ctg.no_alleles_postQC.homozygous_coverage' 
fn = os.path.join(ALLELE_QC_PATH, fn_short)
fh = open(fn, 'w')
for gene in queries_out_filtered:
    print(gene, file=fh)
fh.close()


Out of 550 orphan genes we have 62 orphan buscos and 45 orphan effectors.
On haplotigs. Out of 0 orphan genes we have 0 orphan buscos and 0 orphan effectors.
On pwh. Out of 488 orphan genes we have 57 orphan buscos and 44 orphan effectors.
On pwoh. Out of 62 orphan genes we have 5 orphan buscos and 1 orphan effectors.


In [113]:
#now see if there is an enrichment in GO terms associated with alleles
#now make new filter for the following
out_filter = (tmp_df.exn_asso_contig == True) | (tmp_df.exn_no_asso_contig == True)
queries_out_filtered = tmp_df[out_filter]['Query'].unique()
#now see if there is an enrichment in GO terms associated with alleles
orphan_analysis(queries_out_filtered)
#now print this out again into the ALLELE_QC folder
fn_short = 'Pst_104E_v12_ph_ctg.no_alleles_postQC.exonerate_hits' 
fn = os.path.join(ALLELE_QC_PATH, fn_short)
fh = open(fn, 'w')
for gene in queries_out_filtered:
    print(gene, file=fh)
fh.close()

Out of 816 orphan genes we have 10 orphan buscos and 97 orphan effectors.
On haplotigs. Out of 393 orphan genes we have 5 orphan buscos and 49 orphan effectors.
On pwh. Out of 409 orphan genes we have 5 orphan buscos and 47 orphan effectors.
On pwoh. Out of 14 orphan genes we have 0 orphan buscos and 1 orphan effectors.


In [114]:
len(tmp_df.Query.unique())

4541

In [115]:
#now see if there is an enrichment in GO terms associated with alleles
#now make new filter for the following
out_filter = (tmp_df.exn_asso_contig == True) | (tmp_df.exn_no_asso_contig == True) & \
            (tmp_df.ph_p_homo_region != True)
queries_out_filtered = tmp_df[out_filter]['Query'].unique()
#now see if there is an enrichment in GO terms associated with alleles
orphan_analysis(queries_out_filtered)

Out of 809 orphan genes we have 10 orphan buscos and 96 orphan effectors.
On haplotigs. Out of 393 orphan genes we have 5 orphan buscos and 49 orphan effectors.
On pwh. Out of 404 orphan genes we have 5 orphan buscos and 46 orphan effectors.
On pwoh. Out of 12 orphan genes we have 0 orphan buscos and 1 orphan effectors.


In [116]:
#do filtering of the whole dataframe with QCov and PctID
#now make new filter for the following
out_filter = (allele_QC_df.exn_asso_contig == True) | (allele_QC_df.exn_no_asso_contig == True) 
queries_out_filtered = allele_QC_df[out_filter]['Query'].unique()
#now see if there is an enrichment in GO terms associated with alleles
orphan_analysis(queries_out_filtered)

Out of 2739 orphan genes we have 62 orphan buscos and 240 orphan effectors.
On haplotigs. Out of 1448 orphan genes we have 33 orphan buscos and 126 orphan effectors.
On pwh. Out of 1212 orphan genes we have 27 orphan buscos and 105 orphan effectors.
On pwoh. Out of 79 orphan genes we have 2 orphan buscos and 9 orphan effectors.


Look at some features of the orphan genes such as:
* gene/protein length
* gc content
* numbers of exons
* distance to TEs?

In [118]:
#now look at single haplotype genes
shg_df.drop_duplicates('Query')['QLgth'].mean()

349.0830013280212

In [119]:
allele_blast_df.drop_duplicates('Query')['QLgth'].mean()

394.5913485685585

In [122]:
stats.ranksums(shg_df.drop_duplicates('Query')['QLgth'],allele_blast_df.drop_duplicates('Query')['QLgth'] )

RanksumsResult(statistic=-4.8498250071590965, pvalue=1.2357044232530989e-06)

In [148]:
def len_per_gene(gene_list, gg_fasta):
    '''Returns the len per gene in the list plus the len
    for all genes in the genome from using gg_fasta genome gene fasta.'''
    GC_list_genes = []
    GC_list_genome = []
    for x in SeqIO.parse(gg_fasta, 'fasta'):
        GC_list_genome.append(len(x.seq))
        if x.id.replace('TU', 'model') in gene_list:
            GC_list_genes.append(len(x.seq))
    return GC_list_genes, GC_list_genome

In [149]:
def GC_per_gene(gene_list, gg_fasta):
    '''Returns the GC content per gene in the list plus the GC content
    for all genes in the genome from using gg_fasta genome gene fasta.'''
    GC_list_genes = []
    GC_list_genome = []
    for x in SeqIO.parse(gg_fasta, 'fasta'):
        GC_list_genome.append(SeqUtils.GC(x.seq))
        if x.id.replace('TU', 'model') in gene_list:
            GC_list_genes.append(SeqUtils.GC(x.seq))
    return GC_list_genes, GC_list_genome

In [160]:
def return_gene_fasta(gene_list, gg_fasta):
    '''Returns the fasta of gene in list as SeqIO object '''
    genes_fasta = []
    for x in SeqIO.parse(gg_fasta, 'fasta'):
        if x.id.replace('TU', 'model') in gene_list:
            genes_fasta.append(x)
    return genes_fasta

In [170]:
def return_contig_fasta(contig_list, gg_fasta):
    '''Returns the fasta of contig in list as SeqIO object '''
    contig_fasta = []
    for x in SeqIO.parse(gg_fasta, 'fasta'):
        if x.id in contig_list:
            contig_fasta.append(x)
    return contig_fasta

In [150]:
shg_GC, genome_GC = GC_per_gene(shg, ph_gene_fasta)

In [151]:
#gc content per gene
print(np.mean(shg_GC), np.mean(genome_GC))

45.936932047 45.7863006323


In [152]:
#testing difference
stats.ranksums(shg_GC, genome_GC)

RanksumsResult(statistic=1.7879649018298396, pvalue=0.073781670619541995)

In [154]:
#gene length
shg_len, genome_len = len_per_gene(shg, ph_gene_fasta)

In [155]:
#testing difference
print(np.mean(shg_len), np.mean(genome_len))
stats.ranksums(shg_len, genome_len)

1334.79282869 1538.62967371


RanksumsResult(statistic=-4.8395122876180281, pvalue=1.3015814430474663e-06)

In [164]:
#get gene fastas and save those
shg_fasta = return_gene_fasta(shg, ph_gene_fasta)
shg_fn = os.path.join(OUT_PATH, "Pst_104E_ph_ctg.single_haplotype_genes.fasta")
with open(shg_fn, 'w') as fh:
    SeqIO.write(shg_fasta, fh, 'fasta')

Now pick a couple of the non_respective_haplotype alignements and get sequencing data for those to make a plot

In [311]:
no_respective_alleles_fn = os.path.join(ALLELE_PATH, 'Pst_104E_v12_p_ctg.no_respective_h_contig_overlap.alleles')
contig_fa_fn = os.path.join(BASE_A_PATH, 'Pst_104E_v12_ph_ctg.fa')
gene_fa_fn = os.path.join(BASE_A_PATH, 'Pst_104E_v12_ph_ctg.anno.gene.fa')
OUT_PATH_NO_RES = os.path.join(OUT_PATH, 'no_respect_fasta')
if not os.path.exists(OUT_PATH_NO_RES):
    os.mkdir(OUT_PATH_NO_RES)

In [174]:
no_respective_alleles_df = pd.read_csv(no_respective_alleles_fn, sep='\t', header=None, names=['pgene', 'hgene'])

In [188]:
#get the contig genes are on
no_respective_alleles_df['pcontig'] = no_respective_alleles_df.pgene.str.extract(r'(p[^.]*)')
no_respective_alleles_df['hcontig'] = no_respective_alleles_df.hgene.str.extract(r'(h[^.]*)')

/home/benjamin/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  """Entry point for launching an IPython kernel.
/home/benjamin/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:2: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  


In [290]:
#prepare for filtering on most hit contigs. First by a group by function
no_ra_linkage_count_df = no_respective_alleles_df.groupby(['pcontig','hcontig'])['pgene'].count()
no_ra_linkage_count_max = no_respective_alleles_df.groupby(['pcontig','hcontig'])['pgene']\
            .count().max(level=0)

In [291]:
no_ra_linkage_count_df = no_ra_linkage_count_df.unstack()

In [299]:
p_contig_list = []
most_linked_hcontig = []
number_of_hits = []
for _index, row in no_ra_linkage_count_df.iterrows():
    if row.max() > 2:
        p_contig_list.append(_index)
        most_linked_hcontig.append(row[row==no_ra_linkage_count_max[_index]].index.tolist()[0])
        number_of_hits.append(row[row==no_ra_linkage_count_max[_index]].max())
        #print('primary contig %s and %s overlaps %i' %\
              #(_index, row[row==no_ra_linkage_count_max[_index]].index.tolist()[0],row[row==no_ra_linkage_count_max[_index]].max() ))

In [300]:
most_linked_non_ra_alleles_df = pd.concat([pd.Series(p_contig_list)\
                            , pd.Series(most_linked_hcontig), pd.Series(number_of_hits)], axis=1)

In [301]:
most_linked_non_ra_alleles_df.rename(columns={0:'pcontig', 1:'hcontig', 2:'number_of_overlaps'}, inplace=True)

In [313]:
#now loop over the best hit dataframe to get all the fasta sequences of the repective alleles

for _index, row in most_linked_non_ra_alleles_df.iterrows():
    tmp_p_contig = row['pcontig']
    tmp_h_contig = row['hcontig']
    h_genes = no_respective_alleles_df[(no_respective_alleles_df.pcontig==tmp_p_contig)&\
                                      (no_respective_alleles_df.hcontig==tmp_h_contig)]\
                                        ['hgene'].unique()
    p_genes = no_respective_alleles_df[(no_respective_alleles_df.pcontig==tmp_p_contig)&\
                                      (no_respective_alleles_df.hcontig==tmp_h_contig)]\
                                        ['pgene'].unique()
    #get the h_gene and p_gene sequenes
    h_gene_seq = return_gene_fasta(h_genes, gene_fa_fn)
    p_gene_seq = return_gene_fasta(p_genes, gene_fa_fn)
    fh = open(os.path.join(OUT_PATH_NO_RES, 'no_res_allele_%s_gene.fa' %tmp_p_contig), 'w')
    SeqIO.write(p_gene_seq+h_gene_seq, fh, 'fasta')
    fh.close()
    #get the contig sequences
    p_contig_seq = return_contig_fasta([tmp_p_contig], contig_fa_fn)
    h_contig_seq = return_contig_fasta([tmp_h_contig], contig_fa_fn)
    fh = open(os.path.join(OUT_PATH_NO_RES, 'no_res_allele_%s_contig.fa' %tmp_p_contig), 'w')
    SeqIO.write(p_contig_seq+h_contig_seq, fh, 'fasta')
    fh.close()
    
    

In [314]:
most_linked_non_ra_alleles_df.sort_values('number_of_overlaps')

,pcontig,hcontig,number_of_overlaps
18,pcontig_042,hcontig_000_054,3.0
23,pcontig_086,hcontig_225_001,3.0
17,pcontig_040,hcontig_032_001,3.0
24,pcontig_087,hcontig_072_099,3.0
15,pcontig_032,hcontig_030_011,3.0
25,pcontig_091,hcontig_044_023,3.0
13,pcontig_029,hcontig_012_028,3.0
28,pcontig_104,hcontig_016_024,3.0
29,pcontig_170,hcontig_072_009,3.0
10,pcontig_021,hcontig_000_003,3.0


In [171]:
!ls {BASE_A_PATH}

hcontig_greater_2000.txt	    Pst_104E_v12_p_ctg.fa
hcontig_smaller_2000.txt	    Pst_104E_v12_p_ctg.fa.fai
pcontig_greater_2000.txt	    Pst_104E_v12_p_ctg.genome_file
pcontig_smaller_2000.txt	    Pst_104E_v12_p_ctg.REPET.gff
Pst_104E_v12_h_ctg.anno.CDS.fa	    Pst_104E_v12_ph_ctg.anno.CDS.fa
Pst_104E_v12_h_ctg.anno.CDS.gff3    Pst_104E_v12_ph_ctg.anno.CDS.gff3
Pst_104E_v12_h_ctg.anno.gene.fa     Pst_104E_v12_ph_ctg.anno.gene.fa
Pst_104E_v12_h_ctg.anno.gff3	    Pst_104E_v12_ph_ctg.anno.gff3
Pst_104E_v12_h_ctg.anno.protein.fa  Pst_104E_v12_ph_ctg.anno.protein.fa
Pst_104E_v12_h_ctg.fa		    Pst_104E_v12_ph_ctg.fa
Pst_104E_v12_h_ctg.fa.fai	    Pst_104E_v12_ph_ctg.fa.fai
Pst_104E_v12_h_ctg.genome_file	    Pst_104E_v12_ph_ctg.REPET.gff
Pst_104E_v12_h_ctg.REPET.gff	    Pst_104E_v12_pwh_ctg.anno.gff3
Pst_104E_v12_p_ctg.anno.CDS.fa	    Pst_104E_v12_pwh.txt
Pst_104E_v12_p_ctg.anno.CDS.gff3    Pst_104E_v12_pwoh_ctg.anno.gff3
Pst_104E_v12_p_ctg.anno.gene.fa     Pst_104E_v12_pwoh.txt
Pst_104E_v12_p_ct

In [ ]:
len(tmp_df_ids)

In [ ]:
tmp_df.QCov

In [ ]:
QP_filter = (allele_QC_df.QCov >= QCov_limit) & (allele_QC_df.PctID >= PctID_limit)

In [ ]:
#now a bit more specific for each haplotyp
os.chdir(ALLELE_QC_PATH)
allele_blast_df.head()